In [ ]:
from pathlib import Path

import numpy as np
import scipy as sp
import scipy.stats
import matplotlib.pyplot as plt

import holodeck as holo
from holodeck import utils, cosmo
from holodeck.constants import *

In [ ]:
fname_data = ["bulge-masses", "J_ApJS_210_3_table4.txt"]
fname_data = Path(holo._PATH_DATA).joinpath(*fname_data)
assert fname_data.is_file(), f"Data file {fname_data} does not exist!"

In [ ]:
size_init = 1e6

size_init = int(size_init)
redz = np.zeros(size_init)
zmax = np.zeros(size_init)
# each array is median, 16th, 84th percentile
mtot = np.zeros((size_init, 3))
mdisk = np.zeros((size_init, 3))
mbulge = np.zeros((size_init, 3))
# 1 = 	A de Vaucouleurs component dominates the surface-brightness profile at all radii, or no exponential component is fit (i.e., B/T=1). We also include here galaxies for which the best-fit disk scale length is less than 0.2 pixels. In terms of the single-component Sersic profiles fit by Simard et al., 2011, Cat. J/ApJS/196/11, the distribution of n for these profiles peaks at n∼4-5. These are single-component elliptical galaxies, and are related to profile types 5 and 8 from Allen et al. (2006MNRAS.371....2A).
# 2 = 	The exponential component dominates the surface-brightness profile at all radii, or no de Vaucouleurs component is fit (i.e., B/T=0). As for type 1 profiles, we include here any fits where the half-light radius of the best-fit de Vaucouleurs profile is less than 0.2 pixels (∼2.6% of fits). These galaxies have low n in the single Sersic fits, and are either pure disk systems or disks hosting a weak central pseudo-bulge. These galaxies are related to profile types 2, 5, 7, and 8 from Allen et al. (2006MNRAS.371....2A).
# 3 = 	The de Vaucouleurs component dominates the surface-brightness profile in the central regions, while the exponential component dominates at large radii. The de Vaucouleurs and exponential profiles cross only once at an r-band surface brightness, µr, less than 26mag/arcsec2. For the most part these are genuine bulge+disk systems, however in some cases these are single-component (elliptical) galaxies in which a disk has been included to account for deviations from a pure de Vaucouleurs profile, e.g., tidal features, isophotal twists, or n≳4. Such spurious fits stand out in the distribution of axis ratios as an excess of face-on disks. These are classified as type 1 profiles by Allen et al. (2006MNRAS.371....2A).
# 4 = 	These galaxies include everything that cannot be classified as types 1, 2, or 3. These include galaxies where the bulge and disk surface-brightness profiles cross twice at µr<26mag/arcsec2, or where the disk and bulge profiles are inverted - i.e., the disk profile dominates in the central regions and the bulge dominates at large radii. These encompass profile types 3, 4 and 6 from Allen et al. (2006MNRAS.371....2A). See section 5.2 for further details.
prof_type = np.zeros(size_init, dtype=int)
cnt = 0
with open(fname_data, 'r') as input:
    size = None
    for ii, line in enumerate(input.readlines()):
        line = line.strip()
        if ii < 7:
            continue
        if line.startswith('---'):
            continue
        line = line.split("|")
        if size is None:
            size = len(line)
        elif len(line) != size:
            print(ii, size, len(line))
            raise
        redz[cnt] = float(line[1])
        for jj in range(3):
            mtot[cnt, jj] = float(line[2+jj])
        for jj in range(3):
            mbulge[cnt, jj] = float(line[5+jj])
        for jj in range(3):
            mdisk[cnt, jj] = float(line[8+jj])

        # if mdisk[cnt, 0] > mtot[cnt, 0]:
        #     print(ii, cnt, line)
        #     raise

        zmax[cnt] = float(line[-4])
        prof_type[cnt] = int(line[-2])
        # if cnt < 10:
        #     print(ii, cnt, line)
        #     print(redz[cnt], mtot[cnt, 0], mbulge[cnt, 0], mdisk[cnt, 0], prof_type[cnt])
        cnt += 1

print(f"Loaded {cnt} elements")
redz = redz[:cnt]
zmax = zmax[:cnt]
mtot = mtot[:cnt]
mdisk = mdisk[:cnt]
mbulge = mbulge[:cnt]
prof_type = prof_type[:cnt]

# fill_value = np.nan
fill_value = -np.inf
mtot[mtot < 0.0] = fill_value
mbulge[mbulge < 0.0] = fill_value
mdisk[mdisk < 0.0] = fill_value

In [ ]:
fig, ax = plt.subplots()
ax.set(xlabel='Total Stellar Mass', ylabel='Stellar Bulge Mass', title='Bulge vs. Total')
ax.grid(True, alpha=0.2)
plt.scatter(mtot[:, 0], mbulge[:, 0], alpha=0.2, s=20)
plt.plot([6, 13], [6, 13], 'k--', alpha=0.5)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set(xlabel='Total Stellar Mass', ylabel='Stellar Bulge Mass', title='Bulge vs. Bulge+Disk')
ax.grid(True, alpha=0.2)
temp = np.log10(10.0**mbulge[:, 0] + 10.0**mdisk[:, 0])
plt.scatter(temp, mbulge[:, 0], alpha=0.2, s=20)
plt.plot([6, 13], [6, 13], 'k--', alpha=0.5)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set(xlabel='Total Stellar Mass', ylabel='Stellar Bulge Mass', title='Bulge vs. Bulge+Disk')
ax.grid(True, alpha=0.2)
mb = 10.0**mbulge[:, 0]
md = 10.0**mdisk[:, 0]
mt = 10.0**mtot[:, 0]
temp = np.log10(mb + md)
# temp = np.log10(mt)
frac = mb / (mb + md)
plt.scatter(temp, frac, alpha=0.01, s=20)
# plt.plot([6, 13], [6, 13], 'k--', alpha=0.5)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set(xlabel='Total Stellar Mass', ylabel='Stellar Bulge Mass', title='Bulge vs. Bulge+Disk')
ax.grid(True, alpha=0.2)
mb = 10.0**mbulge[:, 0]
md = 10.0**mdisk[:, 0]
# mt = 10.0**mtot[:, 0]
temp = np.log10(mb + md)
err = np.fabs(temp - mtot[:, 0])
# print(utils.stats(err))
sel = (err < 0.1)
print(utils.frac_str(sel))
frac = mb / (mb + md)
plt.scatter(temp[sel], frac[sel], alpha=0.01, s=20)
# plt.plot([6, 13], [6, 13], 'k--', alpha=0.5)
plt.show()

In [ ]:
fig, ax = plt.subplots()
ax.set(xlabel='Total Stellar Mass', ylabel='Stellar Bulge Mass', title='Bulge vs. Bulge+Disk')
ax.grid(True, alpha=0.2)
mb = 10.0**mbulge[:, 0]
md = 10.0**mdisk[:, 0]
# mt = 10.0**mtot[:, 0]
temp = np.log10(mb + md)
err = np.fabs(temp - mtot[:, 0])
# print(utils.stats(err))
sel = (err < 0.1) & (redz > 0.1)
print(utils.frac_str(sel))
frac = mb / (mb + md)
# plt.hexbin(temp[sel], frac[sel], gridsize=10)
plt.hist2d(temp[sel], frac[sel])
# plt.plot([6, 13], [6, 13], 'k--', alpha=0.5)
plt.show()

In [ ]:
mb = 10.0**mbulge[:, 0]
md = 10.0**mdisk[:, 0]
# mt = 10.0**mtot[:, 0]
temp = np.log10(mb + md)
err = np.fabs(temp - mtot[:, 0])
# print(utils.stats(err))
sel = (err < 0.1)
print(utils.frac_str(sel))
frac = mb / (mb + md)
print(utils.frac_str(sel & (frac > 0.9)))


In [ ]:
NUM_BINS = 30
ERR_MAX = 0.1

mb = 10.0**mbulge[:, 0]
md = 10.0**mdisk[:, 0]
mt = np.log10(mb + md)

err = np.fabs(temp - mtot[:, 0])
sel =  (err < ERR_MAX) & (prof_type == 3) & (redz > 0.2)
print(f"initial selection : {utils.frac_str(sel)}")

mb = mb[sel]
md = md[sel]
mt = mt[sel]
zz = zmax[sel]
frac = mb / (mb + md)

idx = np.argsort(mt)
mb = mb[idx]
md = md[idx]
mt = mt[idx]
zz = zz[idx]
frac = frac[idx]
# weights = np.ones_like(mt)
weights = 1.0 / cosmo.comoving_volume(zz).value

cdf = np.cumsum(weights)
cdf /= cdf[-1]

ybins = np.linspace(0.0, 1.0, NUM_BINS+1)
xbins = np.searchsorted(cdf, ybins)
xbins = mt[xbins]
xbins[0] *= 0.99
xbins[-1] *= 1.01

pp = sp.stats.norm.cdf([0, -2, +2, -1, +1])
yy = np.zeros((NUM_BINS, len(pp)))
for ii in range(NUM_BINS):
    lo = xbins[ii]
    hi = xbins[ii+1]
    sel = (lo < mt) & (mt < hi)
    # yy[ii, :] = np.percentile(frac[sel], 100*pp)
    yy[ii, :] = utils.quantiles(frac[sel], percs=pp, weights=weights[sel])

xx = utils.midpoints(xbins, log=False)
fig, axes = plt.subplots(nrows=2, sharex=True)
plt.subplots_adjust(hspace=0.1)
ax = axes[1]
ax.set(xlabel='Total Stellar Mass [$M_\odot$]', ylabel='Bulge Fraction')
ax.grid(True, alpha=0.15)
cc, = ax.plot(xx, yy[:, 0], '.-')
cc = cc.get_color()
for ii in range(2):
    ax.fill_between(xx, yy[:, 2*ii+1], yy[:, 2*ii+2], color=cc, alpha=0.2)

ax = axes[0]
ax.set(yscale='log', ylabel='Number [$dN/d\log_{10}M_\star$]')
ax.grid(True, alpha=0.15)
ax.hist(mt, bins=xbins, density=True, weights=weights, histtype='step', label='weighted')
ax.hist(mt, bins=xbins, density=True, weights=None, histtype='step', label='unweighted')
ax.legend()

plt.show()